In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import re
from PIL import Image
import requests
import random
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

In [3]:
books = pd.read_csv("Books.csv", low_memory=False)
ratings = pd.read_csv("Ratings.csv", low_memory=False)
users = pd.read_csv("Users.csv", low_memory=False)
print("books : ")
print(books.head(3))
print("ratings : ")
print(ratings.head(3))
print("users : ")
print(users.head(3))

books : 
         ISBN            Book-Title           Book-Author Year-Of-Publication  \
0  0195153448   Classical Mythology    Mark P. O. Morford                2002   
1  0002005018          Clara Callan  Richard Bruce Wright                2001   
2  0060973129  Decision in Normandy          Carlo D'Este                1991   

                 Publisher                                        Image-URL-S  \
0  Oxford University Press  http://images.amazon.com/images/P/0195153448.0...   
1    HarperFlamingo Canada  http://images.amazon.com/images/P/0002005018.0...   
2          HarperPerennial  http://images.amazon.com/images/P/0060973129.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  htt

In [4]:
books_data=books.merge(ratings,on="ISBN")
books_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [6]:
df=books_data.copy()
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.drop(columns=["ISBN","Year-Of-Publication","Image-URL-S","Image-URL-M", "Publisher", "Image-URL-L"],axis=1,inplace=True)
df.drop(index=df[df["Book-Rating"]==0].index,inplace=True)
df["Book-Title"]=df["Book-Title"].apply(lambda x: re.sub("[\W_]+"," ",x).strip())
df.head()

,Book-Title,Book-Author,User-ID,Book-Rating
1,Clara Callan,Richard Bruce Wright,8,5
3,Clara Callan,Richard Bruce Wright,11676,8
5,Clara Callan,Richard Bruce Wright,67544,8
8,Clara Callan,Richard Bruce Wright,116866,9
9,Clara Callan,Richard Bruce Wright,123629,9


In [35]:
# 책 평가 200번 초과한 사람들 -> new_df
new_df=df[df['User-ID'].map(df['User-ID'].value_counts()) > 100]  # Drop users who vote less than 200 times.
users_pivot=new_df.pivot_table(index=["User-ID"],columns=["Book-Title"],values="Book-Rating")
users_pivot.fillna(0,inplace=True)
users_pivot

Book-Title,01 01 00 A Novel of the Millennium,01 01 00 The Novel of the Millennium,1 000 Makers of the Millennium The Men and Women Who Have Shaped the Last 1 000 Years,1 000 Places to See Before You Die,1 000 Years 1 000 People Ranking the Men and Women Who Shaped the Millennium,1 001 Health Tips,1 001 Old Time Garden Tips Timeless Bits of Wisdom on How to Grow Everything Organically from the Good Old Days When Everyone Did,1 003 Great Things About Kids,1 2 3 Magic Effective Discipline for Children 2 12,1 2 3 Music,...,the Devil Will Drag You Under,them Modern Library,together by christmas,wet sand raven tracks,whataboutrick com a poetic tribute to Richard A Ricci,y los 40 ladrones ColecciÃ³n FÃ bula,Â Corre perro corre,Â Eres tu mi mamÃ Are You My Mother,Ã ngeles fugaces Falling Angels,Ã Â ber das Fernsehen
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# 유저의 최고 평점 책 5권 골라 리턴
def users_choice(id):
    
    users_fav=new_df[new_df["User-ID"]==id].sort_values(["Book-Rating"],ascending=False)[0:5]
    return users_fav

In [25]:
users_choice(274301)["Book-Title"]

700387    A Hundred and One Uses of a Dead Cat
586692                         Tea at Gunter s
457586                           Grim Pickings
586664                      Multiple Sclerosis
586665                      Multiple Sclerosis
Name: Book-Title, dtype: object

In [36]:
# cosine similarity 활용해 가장 유사한 유저 5명 찾아 user id 리턴
def user_based(new_df,id):
    if id not in new_df["User-ID"].values:
        print("---일치하는 유저 없습니다---")
        
        
    else:
        index=np.where(users_pivot.index==id)[0][0]
        similarity=cosine_similarity(users_pivot)
        similar_users=list(enumerate(similarity[index]))
        # 0번째 id는 제외(자기 자신임)
        similar_users = sorted(similar_users,key = lambda x:x[1],reverse=True)[1:6]
    
        user_rec=[]
    
        for i in similar_users:
                data=df[df["User-ID"]==users_pivot.index[i[0]]]
                user_rec.extend(list(data.drop_duplicates("User-ID")["User-ID"].values))
                
        cosine_sim_rec = [i[1] for i in similar_users]
    return user_rec, cosine_sim_rec

In [ ]:
# centered cosine
def user_based_centered(new_df, id):

In [50]:
user_id = random.choice(new_df["User-ID"].values)
user_rec, cosine_sim_rec = user_based(new_df,user_id)
print("me:", user_id)
print("sim users id:", user_rec)
print("5 cos sim:", cosine_sim_rec)


me: 30276
sim users id: [114368, 123883, 247447, 227520, 212965]
5 cos sim: [0.07793231204824197, 0.06737350203949756, 0.06489177536779765, 0.05262662661687774, 0.05254663348496708]


In [19]:
# user(userid와 유사한 유저 목록) 와 user_id가 평가한 책들에서 
# userid가 아직 읽지 않은 책 상위 5권 리턴
def common(new_df,user,user_id):
    x=new_df[new_df["User-ID"]==user_id]
    recommend_books=[]
    user=list(user)
    for i in user:
        y=new_df[(new_df["User-ID"]==i)]
        books=y.loc[~y["Book-Title"].isin(x["Book-Title"]),:]
        books=books.sort_values(["Book-Rating"],ascending=False)[0:5]
        recommend_books.extend(books["Book-Title"].values)
        
    return recommend_books[0:5]

In [27]:
user_id = random.choice(new_df["User-ID"].values)
user_choice_df = pd.DataFrame(users_choice(user_id))
user_favorite = users_choice(user_id)
n = len(user_choice_df["Book-Title"].values)
print("🟦 USER: {} ".format(user_id))

# YOUR FAVORITE BOOKS
user_favorite_books = []
for i in range(n):
    book_title = user_choice_df["Book-Title"].tolist()[i]
    rating = round(new_df[new_df["Book-Title"] == book_title]["Book-Rating"].mean(), 1)
    user_favorite_books.append((book_title, rating))

# YOU MAY ALSO LIKE THESE BOOKS
user_based_rec = user_based(new_df, user_id)
books_for_user = common(new_df, user_based_rec, user_id)
recommended_books = []
for i in range(5):
    book_title = books_for_user[i]
    rating = round(new_df[new_df["Book-Title"] == book_title]["Book-Rating"].mean(), 1)
    recommended_books.append((book_title, rating))

# 출력
print("YOUR FAVORITE BOOKS:")
for book, rating in user_favorite_books:
    print(f"Book: {book}, Rating: {rating}")
print("--------------------------------------------------------------------------------------")
print("Recommend these Books!!")
for book, rating in recommended_books:
    print(f"Book: {book}, Rating: {rating}")    
print("--------------------------------------------------------------------------------------")
print("Your best 5 Similar Users")
for id in user_based_rec:
    print("UserId:", id)
    for book in users_choice(id)["Book-Title"]:
        print(book)
    print("----------------------------------------")


🟦 USER: 88677 
YOUR FAVORITE BOOKS:
Book: Love Not a Rebel, Rating: 10.0
Book: Glory Topaz Historical Romance, Rating: 9.5
Book: Surrender Florida Civil War Paperback, Rating: 10.0
Book: Fatal Majesty A Novel of Mary Queen of Scots, Rating: 9.0
Book: Spellbound in Seattle, Rating: 9.0
--------------------------------------------------------------------------------------
Recommend these Books!!
Book: The Ultimate Cross Stitch Companion An Encyclopedia of Techniques and Ideas With over 150 Step By Step Projects, Rating: 10.0
Book: The Outlandish Companion, Rating: 9.0
Book: Velvet Angel, Rating: 9.2
Book: Dragonfly in Amber, Rating: 8.5
Book: Highland Velvet, Rating: 7.5
--------------------------------------------------------------------------------------
Your best 5 Similar Users
UserId: 88677
Love Not a Rebel
Glory Topaz Historical Romance
Surrender Florida Civil War Paperback
Fatal Majesty A Novel of Mary Queen of Scots
Spellbound in Seattle
----------------------------------------
U